# Data Processing

## What is in this file ? (Not now, TODO while needed)

This file contains the code to process the data. 
We will first to load the two datasets, then we will merge them. We will create new feature with One Hot Encoding and wipe previous encoding.

In [19]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from mlxtend.frequent_patterns import apriori, association_rules

## Loading the data

In [67]:
file1 = 'data/globalterrorismdb_2021Jan-June_1222dist.xlsx'
file2 = 'data/globalterrorismdb_0522dist.xlsx'

df1 = pd.read_excel(file1)
df1.shape

(4960, 135)

## Cleaning the dataset


In [68]:
# Colunms to be dropped
## alternative, region, attacktype1, attacktype2, attacktype3, weaptype1, weapsubtype1, weapsubtype3, weaptype4, weapsubtype4, targtype1, 
## targsubtype1, natlty1, targtype2, targsubtype2, natlty2, targtype3,  targsubtype3, natlty3, 
## propextent, claimmode, hostkidoutcome, country

pd.set_option('display.max_columns', None)
df1.drop(columns=['alternative',
                  'region', 
                  'attacktype1', 
                  'attacktype2', 
                  'attacktype3', 
                  'weaptype1', 
                  'weapsubtype1', 
                  'weapsubtype3', 
                  'weaptype4', 
                  'weapsubtype4', 
                  'targtype1', 
                  'targsubtype1', 
                  'natlty1', 
                  'targtype2', 
                  'targsubtype2', 
                  'natlty2', 
                  'targtype3',  
                  'targsubtype3', 
                  'natlty3',
                  'propextent',
                  'claimmode',
                  'hostkidoutcome',
                  'country'
                  ], 
                  inplace=True)
print(df1.shape)
df1.head()

(4960, 112)


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1_txt,attacktype2_txt,attacktype3_txt,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1_txt,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3_txt,weaptype4_txt,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,202101010004,2021,1,1,NaN,0,NaT,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1,0,NaN,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,Bombing/Explosion,NaN,NaN,Military,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Vehicle,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,Remote Trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb was used in ...,2.0,0.0,0.0,5.0,0.0,0.0,1,Unknown,-99.0,Vehicle damaged,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Egyptian officials: Roadside bombing in Sinai...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
1,202101010005,2021,1,1,01/01/2021,0,NaT,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1,0,NaN,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,Bombing/Explosion,NaN,NaN,Police,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),Camp,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,Grenade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Terror module busted, seven arrested in J&K,""...","""Militants throw grenade on CRPF,"" HT Syndicat...",NaN,START Primary Collection,-9,-9,0,-9,NaN
2,202101010006,2021,1,1,NaN,0,NaT,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1,1,NaN,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,Bombing/Explosion,NaN,NaN,Private Citizens & Property,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A mortar was used in the attack.,5.0,0.0,0.0,7.0,0.0,0.0,1,Minor (likely < $1 million),-99.0,Building damaged,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""Civilian reportedly dies of injuries from Hou...","""UN troubled by civilian deaths in Yemen's Hud...","""Five women killed in Yemen wedding attack,"" G...",START Primary Collection,0,0,0,0,NaN
3,202101010009,2021,1,1,NaN,0,NaT,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1,1,NaN,01/01/2021: Assailants opened fire on the vehi...,1,1,1,1,State Actors,0.0,1,0,Assassination,NaN,NaN,Journalists & Media,Radio Journalist/Staff/Facility,Sadai Ghor Radio,Editor: Bismillah Adil Aimaq,Afghanistan

In [69]:
# DATE PREPROCESSING

df1['begin_date'] = pd.to_datetime(df1['iyear'].astype(str) + '-' + df1['imonth'].astype(str) + '-' + df1['iday'].astype(str), errors='coerce', format='%Y-%m-%d')
df1.drop(columns=['iyear', 'imonth', 'iday'], inplace=True)
df1.shape

(4960, 110)

In [70]:
df1['begin_date']

0      2021-01-01
1      2021-01-01
2      2021-01-01
3      2021-01-01
4      2021-01-02
          ...    
4955   2021-06-30
4956   2021-06-30
4957   2021-06-30
4958   2021-06-30
4959   2021-06-30
Name: begin_date, Length: 4960, dtype: datetime64[ns]

In [71]:
cols = df1.columns.tolist()
cols.insert(1, cols.pop(cols.index('begin_date')))
df1 = df1[cols]
df1.head()

,eventid,begin_date,approxdate,extended,resolution,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1_txt,attacktype2_txt,attacktype3_txt,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1_txt,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3_txt,weaptype4_txt,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,202101010004,2021-01-01,NaN,0,NaT,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1,0,NaN,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,Bombing/Explosion,NaN,NaN,Military,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Vehicle,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,Remote Trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb was used in ...,2.0,0.0,0.0,5.0,0.0,0.0,1,Unknown,-99.0,Vehicle damaged,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Egyptian officials: Roadside bombing in Sinai...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
1,202101010005,2021-01-01,01/01/2021,0,NaT,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1,0,NaN,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,Bombing/Explosion,NaN,NaN,Police,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),Camp,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,Grenade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Terror module busted, seven arrested in J&K,""...","""Militants throw grenade on CRPF,"" HT Syndicat...",NaN,START Primary Collection,-9,-9,0,-9,NaN
2,202101010006,2021-01-01,NaN,0,NaT,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1,1,NaN,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,Bombing/Explosion,NaN,NaN,Private Citizens & Property,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A mortar was used in the attack.,5.0,0.0,0.0,7.0,0.0,0.0,1,Minor (likely < $1 million),-99.0,Building damaged,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""Civilian reportedly dies of injuries from Hou...","""UN troubled by civilian deaths in Yemen's Hud...","""Five women killed in Yemen wedding attack,"" G...",START Primary Collection,0,0,0,0,NaN
3,202101010009,2021-01-01,NaN,0,NaT,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1,1,NaN,01/01/2021: Assailants opened fire on the vehi...,1,1,1,1,State Actors,0.0,1,0,Assassination,NaN,NaN,Journalists & Media,Radio Journalist/Staff/Facility,Sadai Ghor Radio,Editor: Bismillah Adil Aimaq,Afghanista

In [72]:
df1[df1['resolution'].notnull()]

df1['end_date'] = pd.to_datetime(df1['resolution'], errors='coerce', format='%Y-%m-%d')

df1.loc[df1['resolution'].isnull(), 'end_date'] = df1['begin_date']


In [73]:
cols = df1.columns.tolist()
cols.insert(2, cols.pop(cols.index('end_date')))
df1 = df1[cols]
df1.head()

,eventid,begin_date,end_date,approxdate,extended,resolution,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1_txt,attacktype2_txt,attacktype3_txt,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1_txt,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3_txt,weaptype4_txt,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,202101010004,2021-01-01,2021-01-01,NaN,0,NaT,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1,0,NaN,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,Bombing/Explosion,NaN,NaN,Military,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Vehicle,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,Remote Trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb was used in ...,2.0,0.0,0.0,5.0,0.0,0.0,1,Unknown,-99.0,Vehicle damaged,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Egyptian officials: Roadside bombing in Sinai...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
1,202101010005,2021-01-01,2021-01-01,01/01/2021,0,NaT,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1,0,NaN,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,Bombing/Explosion,NaN,NaN,Police,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),Camp,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,Grenade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Terror module busted, seven arrested in J&K,""...","""Militants throw grenade on CRPF,"" HT Syndicat...",NaN,START Primary Collection,-9,-9,0,-9,NaN
2,202101010006,2021-01-01,2021-01-01,NaN,0,NaT,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1,1,NaN,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,Bombing/Explosion,NaN,NaN,Private Citizens & Property,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A mortar was used in the attack.,5.0,0.0,0.0,7.0,0.0,0.0,1,Minor (likely < $1 million),-99.0,Building damaged,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""Civilian reportedly dies of injuries from Hou...","""UN troubled by civilian deaths in Yemen's Hud...","""Five women killed in Yemen wedding attack,"" G...",START Primary Collection,0,0,0,0,NaN
3,202101010009,2021-01-01,2021-01-01,NaN,0,NaT,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1,1,NaN,01/01/2021: Assailants opened fire on the vehi...,1,1,1,1,State Actors,0.0,1,0,Assassination,NaN,NaN,Journalists & Media,Radio Journalist/Staff/Facility,Sad

In [74]:
df1.head()

,eventid,begin_date,end_date,approxdate,extended,resolution,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1_txt,attacktype2_txt,attacktype3_txt,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1_txt,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3_txt,weaptype4_txt,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,202101010004,2021-01-01,2021-01-01,NaN,0,NaT,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1,0,NaN,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,Bombing/Explosion,NaN,NaN,Military,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Vehicle,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,Remote Trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb was used in ...,2.0,0.0,0.0,5.0,0.0,0.0,1,Unknown,-99.0,Vehicle damaged,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Egyptian officials: Roadside bombing in Sinai...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
1,202101010005,2021-01-01,2021-01-01,01/01/2021,0,NaT,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1,0,NaN,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,Bombing/Explosion,NaN,NaN,Police,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),Camp,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,Grenade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Terror module busted, seven arrested in J&K,""...","""Militants throw grenade on CRPF,"" HT Syndicat...",NaN,START Primary Collection,-9,-9,0,-9,NaN
2,202101010006,2021-01-01,2021-01-01,NaN,0,NaT,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1,1,NaN,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,Bombing/Explosion,NaN,NaN,Private Citizens & Property,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A mortar was used in the attack.,5.0,0.0,0.0,7.0,0.0,0.0,1,Minor (likely < $1 million),-99.0,Building damaged,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""Civilian reportedly dies of injuries from Hou...","""UN troubled by civilian deaths in Yemen's Hud...","""Five women killed in Yemen wedding attack,"" G...",START Primary Collection,0,0,0,0,NaN
3,202101010009,2021-01-01,2021-01-01,NaN,0,NaT,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1,1,NaN,01/01/2021: Assailants opened fire on the vehi...,1,1,1,1,State Actors,0.0,1,0,Assassination,NaN,NaN,Journalists & Media,Radio Journalist/Staff/Facility,Sad

In [75]:
df1.drop(columns=['resolution', 'approxdate', 'extended'], inplace=True)

In [76]:
df1

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1_txt,attacktype2_txt,attacktype3_txt,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1_txt,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3_txt,weaptype4_txt,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,202101010004,2021-01-01,2021-01-01,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1,0,NaN,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,Bombing/Explosion,NaN,NaN,Military,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Vehicle,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,Remote Trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb was used in ...,2.0,0.0,0.0,5.0,0.0,0.0,1,Unknown,-99.0,Vehicle damaged,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Egyptian officials: Roadside bombing in Sinai...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
1,202101010005,2021-01-01,2021-01-01,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1,0,NaN,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,Bombing/Explosion,NaN,NaN,Police,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),Camp,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,Grenade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Terror module busted, seven arrested in J&K,""...","""Militants throw grenade on CRPF,"" HT Syndicat...",NaN,START Primary Collection,-9,-9,0,-9,NaN
2,202101010006,2021-01-01,2021-01-01,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1,1,NaN,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,Bombing/Explosion,NaN,NaN,Private Citizens & Property,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosives,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A mortar was used in the attack.,5.0,0.0,0.0,7.0,0.0,0.0,1,Minor (likely < $1 million),-99.0,Building damaged,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""Civilian reportedly dies of injuries from Hou...","""UN troubled by civilian deaths in Yemen's Hud...","""Five women killed in Yemen wedding attack,"" G...",START Primary Collection,0,0,0,0,NaN
3,202101010009,2021-01-01,2021-01-01,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1,1,NaN,01/01/2021: Assailants opened fire on the vehi...,1,1,1,1,State Actors,0.0,1,0,Assassination,NaN,NaN,Journalists & Media,Radio Journalist/Staff/Facility,Sadai Ghor Radio,Editor: Bismillah Adil Aimaq,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN